### Generate a recommendation model based on content filtering in python
### At least use the description field. You can decide whether to combine the decription with other fields. You decide.
### Save this model in a .sav file

In [1]:
import pandas as pd

articles_df = pd.read_csv("data/shared_articles.csv")

display(articles_df.head())

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [2]:
articles_df.isna().sum()

timestamp             0
eventType             0
contentId             0
authorPersonId        0
authorSessionId       0
authorUserAgent    2442
authorRegion       2442
authorCountry      2442
contentType           0
url                   0
title                 0
text                  0
lang                  0
dtype: int64

In [3]:
articles_df.dropna(inplace=True)
articles_df = articles_df[articles_df['lang'] == 'en']

articles_df.isna().sum()

timestamp          0
eventType          0
contentId          0
authorPersonId     0
authorSessionId    0
authorUserAgent    0
authorRegion       0
authorCountry      0
contentType        0
url                0
title              0
text               0
lang               0
dtype: int64

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

#Step 1: Import/build your stop word dictionary

#Step 2: Create a TfidfVectorizer and remove stop words
tfidf = TfidfVectorizer(stop_words='english')

articles_df['combined'] = articles_df['title'] + " " + articles_df['text'] #combine features for better accuracy

#Step 3: Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(articles_df['combined'])

print(tfidf_matrix)

#to help us interpret the results, view it in a DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=articles_df['title'])


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 136919 stored elements and shape (473, 19470)>
  Coords	Values
  (0, 9274)	0.009554934467554729
  (0, 5503)	0.4155979893551253
  (0, 19156)	0.6926633155918757
  (0, 15609)	0.21224278343607936
  (0, 256)	0.03528170872174988
  (0, 2336)	0.023983049355927315
  (0, 17692)	0.01078797255248374
  (0, 11031)	0.11964713779648416
  (0, 1289)	0.011836389558532352
  (0, 7535)	0.007285044294131887
  (0, 1815)	0.011157045936076741
  (0, 8643)	0.016004858638758793
  (0, 3926)	0.009554934467554729
  (0, 1662)	0.015386428366421175
  (0, 5120)	0.05051581775651578
  (0, 13712)	0.10772326783847641
  (0, 6450)	0.020841327992781038
  (0, 11703)	0.014736607367102528
  (0, 6814)	0.007534330962593222
  (0, 4099)	0.20082682684938136
  (0, 13315)	0.010611874004709108
  (0, 13273)	0.013539668462971721
  (0, 5090)	0.016305522469861677
  (0, 17341)	0.008364424811094293
  (0, 9043)	0.011994801008154084
  :	:
  (471, 4575)	0.043466842279494856
  (471, 1468

In [5]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df_results = pd.DataFrame(cosine_sim, index=articles_df['title'], columns=articles_df['title'])

df_results


#store df_results as a csv file. open it and sort it by the show you're interested in and the next 5 rows are the most closely related shows.
# df_results.sort_values(by=['title'], ascending=False)

title,Introducing Docker for Windows Server 2016 - Docker Blog,Microsoft and Bank of America Merrill Lynch Collaborate to Transform Trade Finance Transacting With Azure Blockchain as a Service,Digitization and The Loss of Iconography - Posted by SYPartners,"Google saves Allo conversations, a win for machine learning but a loss for privacy - Tech2","A Neural Network for Machine Translation, at Production Scale","Onion Creates a $5 Linux Computer with Wi-Fi, Designed for IoT Applications",​Kubernetes 1.4: One DevOps tool to rule all the containers | ZDNet,Today in Apple history: 'Here's to the crazy ones...' | Cult of Mac,Baidu Research grills AI models on deep learning,Your health insurance might score you an Apple Watch,Bitcoin Accepted! German Energy Giant Enables Payments - CCN: Financial Bitcoin & Cryptocurrency News,Not Your Father's Insurance Industry - Insurance Thought Leadership,Don't Waste Your Time on Networking Events,Digital development with online check-in and mobile room keys,Blockchain Will Be Used By 15% of Big Banks By 2017,How CIOs Leverage Business Intelligence to Drive Smarter Decision-Making,Six iconic retailers and their digital transformation journeys,The future of traditional retail is digital,Large CPGs are under attack by startups... and consumers are winning,Breaking: Apple's Search Ads go LIVE!,"Use Docker to build, test and push your Artifacts",IBM Watson: Talking To An Ad,Bringing Pokémon GO to life on Google Cloud,Google's Cloud Machine Learning service is now in public beta,All Together Now. Introducing G Suite.,Save time with Quick Access in Drive,Google Cloud Platform sets a course for new horizons,New Early Adopter Programs: Team Drives and a new Hangouts video meetings experience,"Explore in Docs, Sheets and Slides makes work a breeze - and makes you look good, too",Tracking Multiple Categories in Google Analytics for Content Pages - E-Nor | Google Analytics Blog,Pury - New Way to Profile Your Android Application,"Facebook, Amazon, Google, IBM and Microsoft come together to create historic Partnership on AI","Largest botnet attack in history peaks at over 1 terabits per second, aided by Internet of Things",The new tech talent you need to succeed in digital,Blog | Niantic,Innovation is in all the wrong places,Introducing the Open Images Dataset,The Best Advice From Quora on 'How to Learn Machine Learning',So you want to sell to banks?,Most experts say AI isn't as much of a threat as you might think,Engineers Shouldn't Write ETL: A Guide to Building a High Functioning Data Science Department,"Meerkat, star app of 2015, is officially dead",Advanced Android Espresso,JPMorgan is Quietly Developing a Private Ethereum Blockchain - CoinDesk,"Simple rolls out shared bank accounts that work for anyone, including roommates",New APIs Expand Line's Third Party Integration and Chatbot Capabilities,Use Poka-Yoke Technique to Improve Software Quality,Future Of Consulting: Consultancies Adapt To Digital Change,How Robots Can Acquire New Skills from Their Shared Experience,New Trend Briefing from TrendWatching | AUTONOMANIA!,...,Centrally manage all your Google Cloud resources with Cloud Resource Manager,Expanding our IDE support with a new Eclipse plugin for App Engine,Guest post: building IoT applications with MQTT and Google Cloud Pub/Sub,Behind the Scam: What Does It Take to Be a 'Best-Selling Author'? $3 and 5 Minutes. - The Mission,BACK FROM THE DEAD - TrendWatching,Speed Up a WordPress Website in 8 Steps,The Complete Google Analytics Audit Checklist | Optimization Up,Java 8 Streams - A Deeper Approach About Performance Improvement,Tom Brady: Business Guru. Who Knew?,How to get the most from your agency relationships in 2017,Introducing Google Cloud Search: Bringing the power of Google Search to G Suite customers,Gmail API: New endpoints for settings,Here's how Evernote moved 3 petabytes of data to Google's cloud,Sharing innovation with your competitors - Dries Buytaert,Python - It's metaclasses all the

In [6]:
df_results.to_csv("../IS 455/content_filtering.csv", index=False)